# Chapter 06: Money management

📈Join our community: https://discord.gg/wXjNPAc5BH

📚Read our book: https://www.amazon.com/gp/product/B09HG18CYL 

🖥️Quantreo's YouTube channel: https://www.youtube.com/channel/UCp7jckfiEglNf_Gj62VR0pw

In [1]:
# Import the libraries
import MetaTrader5 as mt5
from datetime import datetime
import numpy as np

In [2]:
# Create the bound between MT5 and Python
mt5.initialize()

True

### Place an order with take profit & stop loss

In [7]:
# Place a BUY order with take profit
# Initialization value
lot = 0.01
symbol = "EURUSD"

# Extract symbol point
point = mt5.symbol_info(symbol).point

# Choose the deviation
deviation = 10

# Find the filling mode of symbol
filling_type = mt5.symbol_info(symbol).filling_mode

# Create dictionnary request
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_BUY,
    "price": mt5.symbol_info_tick(symbol).ask,
    "deviation": deviation,
    "tp": mt5.symbol_info_tick(symbol).ask + 100*point,
    "sl": mt5.symbol_info_tick(symbol).ask - 100*point, 
    "type_filling": filling_type,
    "type_time": mt5.ORDER_TIME_GTC,
}

mt5.order_check(request).comment

'Done'

In [7]:
# Place a SELL order with take profit
# Initialization value
lot = 0.01
symbol = "EURUSD"

# Extract symbol point
point = mt5.symbol_info(symbol).point

# Choose the deviation
deviation = 10

# Find the filling mode of symbol
filling_type = mt5.symbol_info(symbol).filling_mode

# Create dictionnary request
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_SELL,
    "price": mt5.symbol_info_tick(symbol).bid,
    "deviation": deviation,
    "tp": mt5.symbol_info_tick(symbol).ask - 100 * point,
    "sl": mt5.symbol_info_tick(symbol).ask + 100 * point, 
    "type_filling": filling_type,
    "type_time": mt5.ORDER_TIME_GTC,
}

mt5.order_check(request).comment

'Done'

### Find stop loss and take profit level for a specific risk percentage

In [5]:
def risk_reward_threshold(symbol, buy=True, risk=0.01, reward=0.02):
    
    # Extract the leverage
    leverage = mt5.account_info().leverage

    # Compute the price
    price = mt5.symbol_info(symbol).ask

    # Extract the number of decimals
    nb_decimal = str(price)[::-1].find(".")


    # Compute the variations in percentage
    var_down = risk/leverage
    var_up = reward/leverage


    # Find the TP and SL threshold in absolute price
    if buy:
        price = mt5.symbol_info(symbol).ask

        # Compute the variations in absolute price
        price_var_down = var_down*price
        price_var_up = var_up * price

        tp = np.round(price + price_var_up, nb_decimal)
        sl = np.round(price - price_var_down, nb_decimal)

    else:

        price = mt5.symbol_info(symbol).bid

        # Compute the variations in absolute price
        price_var_down = var_down*price
        price_var_up = var_up * price

        tp = np.round(price - price_var_up, nb_decimal)
        sl = np.round(price + price_var_down, nb_decimal)


        
    print(f"PRICE: {price} \t Take Profit: {tp} \t Stop Loss: {sl}")

In [6]:
risk_reward_threshold("EURUSD",buy=True)

PRICE: 1.1015 	 Take Profit: 1.1022 	 Stop Loss: 1.1011


In [18]:
(1.09682 - 1.09693) / 1.09693 *100 * 100

-1.002798720064676

### Find the volume depending of your capital

In [28]:
def position_size(capital, symbol):
    mt5.initialize()
    print(f"INVESTED CAPITAL: {capital}")
    
    leverage = mt5.account_info().leverage
    print(f"LEVERAGE: {leverage}")
    
    invested_capital = capital  * leverage
    print(f"INVESTED CAPITAL LEVERAGED: {invested_capital}")
    
    trade_size = mt5.symbol_info(symbol).trade_contract_size
    print(f"TRADE SIZE: {trade_size}")
    
    price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid)/2
    print(f"PRICE: {price}")

    lot_size = invested_capital / trade_size / price
    print(f"LOT SIZE: {lot_size}")
    
    min_lot = mt5.symbol_info(symbol).volume_min
    print(f"MIN LOT: {min_lot}")
    
    max_lot = mt5.symbol_info(symbol).volume_max
    print(f"MAX LOT: {max_lot}")


    if min_lot<lot_size:
        number_decimal = str(min_lot)[::-1].find(".")
        print(f"NUMBER DECIMAL: {number_decimal}")

        if number_decimal>0:
            lot_size_rounded = np.round(lot_size, number_decimal)
            print(f"LOT SIZE ROUNDED: {lot_size_rounded}")

            if lot_size < lot_size_rounded:
                lot_size_rounded = np.round(lot_size_rounded - min_lot, number_decimal)
                print(f"LOT DOWN ROUNDED: {lot_size_rounded}")

        else:
            number_size_lot =  len(str(min_lot))

            lot_size_rounded = int(np.round(lot_size, -number_size_lot))

            if lot_size < lot_size_rounded:
                lot_size_rounded = int(np.round(lot_size_rounded - number_size_lot, - number_size_lot))
                
        if lot_size_rounded>max_lot:
            lot_size_rounded = max_lot
            
        print(f"GOOD SIZE LOT: {lot_size_rounded}")
        return lot_size_rounded
    else: 
        print("Invested capital is too small to be able to place an order")
        
    
    
position_size(120, "SP500m")

INVESTED CAPITAL: 120
LEVERAGE: 100
INVESTED CAPITAL LEVERAGED: 12000
TRADE SIZE: 10.0
PRICE: 4312.75
LOT SIZE: 0.27824473943539507
MIN LOT: 0.1
MAX LOT: 5.0
NUMBER DECIMAL: 1
LOT SIZE ROUNDED: 0.3
LOT DOWN ROUNDED: 0.2
GOOD SIZE LOT: 0.2


0.2